# ABC-анализ онлайн-магазина (ДЗ в рамках курса Scypro "Аналитик данных")


# 🎈🍾 🎉





Вы работаете в крупном онлайн-магазине, который реализует товары «всё для праздников». Вам необходимо провести ABC-анализ и выявить наиболее важные товары на основе объемов продаж и популярности. Эта информация составит план закупок на весь следующий год. 

**ABC-анализ** позволяет определить наиболее важные ресурсы компании на основе объемов продаж и прибыли.

В основе ABC-анализа лежит __«Принцип Парето»: 20% ресурсов приносят 80% прибыли__. Именно этот вид анализа позволяет выделить те самые  группы товаров, ресурсов, клиентов, поставщиков и т. п., которые и приносят компании основную прибыль.

__Задача анализа разделить товары на группы по принципу:__

- А — наиболее ценные, 20% — ассортимента (номенклатура); 80% — продаж
- В — промежуточные, 30% — ассортимента; 15% — продаж
- С — наименее ценные, 50% — ассортимента; 5% — продаж

## Задание 1

__Сохраните все данные из таблицы `products` из PostgreSQL в дата-фрейм `products` и подготовьте данные к анализу.__

In [1]:
from postgresql_secret import pg_secret

In [2]:
import psycopg2

connection = psycopg2.connect(
                             database=pg_secret['db_name'],
                             user=pg_secret['db_user'],
                             password=pg_secret['db_password'],
                             host=pg_secret['db_host'],
                             port=pg_secret['db_port'],
                             )

cursor = connection.cursor()

In [3]:
import pandas as pd

query = """SELECT * 
           FROM products"""

cursor.execute(query)            # Сделаем запрос
result = cursor.fetchall()  # Запросим все строки, получим результат в виде списка кортежей

#cursor.description #узнаем имена колонок


columns = [] # В этот список будем сохранять названия колонок
for desc in cursor.description:  
    columns.append(desc[0])      # Названия колонок возьмем из первого элемента cursor.description

    
products = pd.DataFrame(result, columns=columns)  # Из списка кортежей сделаем датафрейм
products.head()

,invoice_id,stock_code,description,quantity,price
0,563031,20725,LUNCH BAG RED RETROSPOT,2,1.65
1,563031,20728,LUNCH BAG CARS BLUE,2,1.65
2,563031,23207,LUNCH BAG ALPHABET DESIGN,2,1.65
3,563031,84596F,SMALL MARSHMALLOWS PINK BOWL,2,0.42
4,563031,84596B,SMALL DOLLY MIX DESIGN ORANGE BOWL,2,0.42


In [4]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 779495 entries, 0 to 779494
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   invoice_id   779495 non-null  int64 
 1   stock_code   779495 non-null  object
 2   description  779495 non-null  object
 3   quantity     779495 non-null  int64 
 4   price        779495 non-null  object
dtypes: int64(2), object(3)
memory usage: 29.7+ MB


In [5]:
#меняем типы данных
products['stock_code'] = products['stock_code'].astype('str')
products['description'] = products['description'].astype('str')
products['price'] = products['price'].astype('float64')

In [6]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 779495 entries, 0 to 779494
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   invoice_id   779495 non-null  int64  
 1   stock_code   779495 non-null  object 
 2   description  779495 non-null  object 
 3   quantity     779495 non-null  int64  
 4   price        779495 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 29.7+ MB


## Задание 2

__Посчитайте, какой доход принес каждый продукт и сколько единиц каждого продукта было продано__


In [7]:
#посчитаем стоимость
products['total'] = products['price']*products['quantity']

In [8]:
# посчитаем общий доход по каждому товару. Поскольку товары могут иметь одинаковое название, 
#то будем группировать и по названию и по id
prod_sum = products.groupby(['description','stock_code'],as_index=False)['total'].sum()
prod_sum.head()
#mTable_sum.info()

,description,stock_code,total
0,DOORMAT UNION JACK GUNS AND ROSES,21955,1071.25
1,3 STRIPEY MICE FELTCRAFT,22150,1241.10
2,4 PURPLE FLOCK DINNER CANDLES,72800B,495.16
3,50'S CHRISTMAS GIFT BAG LARGE,23437,2272.25
4,ANIMAL STICKERS,16011,80.85


In [9]:
# посчитаем общее количество проданных единиц по каждому товару
prod_count = products.groupby(['description','stock_code'], as_index=False)['quantity'].sum()
prod_count.head()
#mTable_count.info()

,description,stock_code,quantity
0,DOORMAT UNION JACK GUNS AND ROSES,21955,167
1,3 STRIPEY MICE FELTCRAFT,22150,662
2,4 PURPLE FLOCK DINNER CANDLES,72800B,324
3,50'S CHRISTMAS GIFT BAG LARGE,23437,1885
4,ANIMAL STICKERS,16011,385


In [10]:
# объединяем общие данные в одну таблицу
prod =  pd.merge(prod_sum, prod_count, how='left', left_on=['description','stock_code'], right_on = ['description','stock_code'])
prod.head()
#mTable.info()

,description,stock_code,total,quantity
0,DOORMAT UNION JACK GUNS AND ROSES,21955,1071.25,167
1,3 STRIPEY MICE FELTCRAFT,22150,1241.10,662
2,4 PURPLE FLOCK DINNER CANDLES,72800B,495.16,324
3,50'S CHRISTMAS GIFT BAG LARGE,23437,2272.25,1885
4,ANIMAL STICKERS,16011,80.85,385


## Задание 3

__Найди нарастающий итог по процентам__



In [11]:
# отсортируем дата-фрейм по убыванию выручки и количества товаров
#prod.sort_values(['total', 'quantity'], ascending=[False, False], inplace=True)

#отсортируем дата-фрейм по доходу
prod.sort_values('total', ascending= False, inplace=True)
prod.head()

,description,stock_code,total,quantity
3789,REGENCY CAKESTAND 3 TIER,22423,277656.25,24139
5079,WHITE HANGING HEART T-LIGHT HOLDER,85123A,247048.01,91757
3145,"PAPER CRAFT , LITTLE BIRDIE",23843,168469.60,80995
2876,Manual,M,151777.67,9391
2405,JUMBO BAG RED RETROSPOT,85099B,134307.44,74224


In [12]:
# найдем процент дохода для каждого товара 
prod['percentage_total'] = 100*prod['total'] /  prod['total'].sum()
prod.head()

,description,stock_code,total,quantity,percentage_total
3789,REGENCY CAKESTAND 3 TIER,22423,277656.25,24139,1.598040
5079,WHITE HANGING HEART T-LIGHT HOLDER,85123A,247048.01,91757,1.421875
3145,"PAPER CRAFT , LITTLE BIRDIE",23843,168469.60,80995,0.969620
2876,Manual,M,151777.67,9391,0.873550
2405,JUMBO BAG RED RETROSPOT,85099B,134307.44,74224,0.773001


In [13]:
# рассчитаем кумулятивной суммы
prod['running_total']  = prod['percentage_total'].cumsum()
prod.head()

,description,stock_code,total,quantity,percentage_total,running_total
3789,REGENCY CAKESTAND 3 TIER,22423,277656.25,24139,1.598040,1.598040
5079,WHITE HANGING HEART T-LIGHT HOLDER,85123A,247048.01,91757,1.421875,3.019915
3145,"PAPER CRAFT , LITTLE BIRDIE",23843,168469.60,80995,0.969620,3.989535
2876,Manual,M,151777.67,9391,0.873550,4.863085
2405,JUMBO BAG RED RETROSPOT,85099B,134307.44,74224,0.773001,5.636086


## Задание 4

__Пометь товары метками `'A'`, `'B'`, `'C'` в зависимости от нарастающего итога__


In [14]:
def revenue(x):
    if x <= 80:
        return 'A'
    elif 80 < x <= 95:
        return 'B'
    else:
        return 'C'

prod['revenue'] = prod['running_total'].apply(revenue)

In [15]:
prod.head()

,description,stock_code,total,quantity,percentage_total,running_total,revenue
3789,REGENCY CAKESTAND 3 TIER,22423,277656.25,24139,1.598040,1.598040,A
5079,WHITE HANGING HEART T-LIGHT HOLDER,85123A,247048.01,91757,1.421875,3.019915,A
3145,"PAPER CRAFT , LITTLE BIRDIE",23843,168469.60,80995,0.969620,3.989535,A
2876,Manual,M,151777.67,9391,0.873550,4.863085,A
2405,JUMBO BAG RED RETROSPOT,85099B,134307.44,74224,0.773001,5.636086,A


## Задание 5

__Найди нарастающий итог по количеству товарных единиц и пометь товары метками `'A'`, `'B'`, `'C'`, отмечая тем самым популярность товара__


In [16]:
#отсортируем дата-фрейм по количеству товаров
prod.sort_values('quantity', ascending= False, inplace=True)

# найдем процент количества для каждого товара 
prod['percentage_quantity'] = 100*prod['quantity'] /  prod['quantity'].sum()

#рассчитаем кумулятивную сумму
prod['running_quantity']  = prod['percentage_quantity'].cumsum()
prod.head()


,description,stock_code,total,quantity,percentage_total,running_total,revenue,percentage_quantity,running_quantity
5200,WORLD WAR 2 GLIDERS ASSTD DESIGNS,84077,24098.03,105185,0.138695,29.646558,A,0.999031,0.999031
5079,WHITE HANGING HEART T-LIGHT HOLDER,85123A,247048.01,91757,1.421875,3.019915,A,0.871494,1.870524
3145,"PAPER CRAFT , LITTLE BIRDIE",23843,168469.60,80995,0.969620,3.989535,A,0.769278,2.639802
293,ASSORTED COLOUR BIRD ORNAMENT,84879,124351.86,78234,0.715702,7.069195,A,0.743054,3.382857
2708,MEDIUM CERAMIC TOP STORAGE JAR,23166,81416.73,77916,0.468591,8.132230,A,0.740034,4.122891


In [17]:
def quantity(x):
    if x <= 80:
        return 'A'
    elif 80 < x <= 95:
        return 'B'
    else:
        return 'C'

prod['popularity'] = prod['running_quantity'].apply(quantity)

In [18]:
prod.head()

,description,stock_code,total,quantity,percentage_total,running_total,revenue,percentage_quantity,running_quantity,popularity
5200,WORLD WAR 2 GLIDERS ASSTD DESIGNS,84077,24098.03,105185,0.138695,29.646558,A,0.999031,0.999031,A
5079,WHITE HANGING HEART T-LIGHT HOLDER,85123A,247048.01,91757,1.421875,3.019915,A,0.871494,1.870524,A
3145,"PAPER CRAFT , LITTLE BIRDIE",23843,168469.60,80995,0.969620,3.989535,A,0.769278,2.639802,A
293,ASSORTED COLOUR BIRD ORNAMENT,84879,124351.86,78234,0.715702,7.069195,A,0.743054,3.382857,A
2708,MEDIUM CERAMIC TOP STORAGE JAR,23166,81416.73,77916,0.468591,8.132230,A,0.740034,4.122891,A


## Задание 6

__Объедини группы и проанализируй__


In [19]:
prod['group'] = prod['revenue'] + prod['popularity'] #создаем серию с с объединенным столбцом отметок популярности и доходности товаров
prod['group'].value_counts() #считаем количество вхождений каждой группы

CC    2419
AA     776
BB     636
BC     412
CB     327
BA     319
AB     288
AC      74
CA      64
Name: group, dtype: int64

# **Выводы**
Чаще всего встречаются группы товаров, которые приносят мало дохода и наименее популярны (группа СС).

При этом на втором месте по частоте - группы товаров, которые приносят основной доход и наиболее популярны (группа AA)


*На каких группах товаров нужно сфокусироваться при составлении плана закупок?*

Наверное, на тех, которые наиболее популярны и приносят основной доход (группа AA)

## Задание 7

__Выгрузи данные по товарам с меткой по ABC-анализу__


In [20]:
abc_analysis_merged = pd.merge(prod, 
                     products, 
                     on='stock_code',  
                     how='inner')
abc_analysis_merged.rename(columns={"group": "ABC-код", "description_x":"Описание", "invoice_id":"ID"}, inplace = True)

abc_analysis = abc_analysis_merged[['ABC-код', 'Описание', 'ID']]
abc_analysis.head()


,ABC-код,Описание,ID
0,AA,WORLD WAR 2 GLIDERS ASSTD DESIGNS,563073
1,AA,WORLD WAR 2 GLIDERS ASSTD DESIGNS,563112
2,AA,WORLD WAR 2 GLIDERS ASSTD DESIGNS,563238
3,AA,WORLD WAR 2 GLIDERS ASSTD DESIGNS,563427
4,AA,WORLD WAR 2 GLIDERS ASSTD DESIGNS,563437


In [21]:
abc_analysis.to_csv('abc_analysis.csv', index=False)